In [ ]:
#Alternative approach
!pip install pyspark

In [ ]:
!pip install findspark

In [ ]:
import os
import sys
import pyspark
import json
import pandas as pd
import gzip
import numpy as np
import numpy as np
import re

In [ ]:
#extract data by start and end
def extraxtString(string,startString,endString):
    startingPoint=0
    lengthString=len(string)
    myList = []
    continueYes = 1
    while(continueYes == 1 and startingPoint < lengthString):
        temp1=string.find(startString,startingPoint)
        if(temp1 != -1):
            temp1 = temp1+len(startString)
            temp2 = string.find(endString,temp1)
            if(temp2 != -1):
                #middleString=string[temp1:temp2]
                middleString=string[(temp1-len(startString)):temp2]
                myList.append(middleString)
                #startingPoint=temp2+len(endString)
                startingPoint=temp2
            else:
                continueYes=0
        else:
            continueYes=0
    return myList

In [ ]:
import findspark
findspark.init()
from pyspark.sql.functions import from_json, udf, col, lower, regexp_replace, split, when
from pyspark.sql.functions import rand #additional library for optimization
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, BooleanType
import random
import json
import re
import string
import time


In [ ]:
!pip install delta

  Preparing metadata (setup.py) ... done
  Created wheel for delta: filename=delta-0.4.2-py3-none-any.whl size=2916 sha256=97e63f790cabf6d1c08acec003634fe787cc4e0d5de8cd60db79489c0e80a0e5
  Stored in directory: /root/.cache/pip/wheels/a8/86/24/a486f14769cf86a2a9ce6b589a82b7414b14657c6fd515dc75
Successfully built delta


In [ ]:
!pip install matplotlib-venn

In [ ]:
##############################################################
##############################################################

In [ ]:
import findspark
findspark.init()
from pyspark.sql.functions import from_json, udf, col, lower, regexp_replace, split, when
from pyspark.sql.functions import rand #additional library for optimization
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, BooleanType
import random
import json
import re
import string
import time

from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

In [ ]:
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.base import Transformer
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession, DataFrame
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.types import IntegerType, StringType, ArrayType
from pyspark.sql.functions import when

In [ ]:
#Added last two .config to allocate memory and ensure 4 cores to be utilized
#Set memory per exector to 2g to be able to handle the 5GB file
builder = SparkSession.builder.appName("amazon") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.executor.cores", "3") \
    .config("spark.executor.instances", "4") \
    .config("spark.executor.memory", "2g")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
#Create a dataframe from the txt file by reading it as json for formating
df = spark.read.json("/content/drive/MyDrive/GoogleColab/Video_Games_5.json.gz")

In [ ]:
df.show()

+----------+-----+-------+--------------------+-----------+--------------+-----------------+-----+--------------------+--------------+--------+----+
|      asin|image|overall|          reviewText| reviewTime|    reviewerID|     reviewerName|style|             summary|unixReviewTime|verified|vote|
+----------+-----+-------+--------------------+-----------+--------------+-----------------+-----+--------------------+--------------+--------+----+
|0700026657| NULL|    5.0|This game is a bi...|10 17, 2015|A1HP7NVNPFMA4N|      Ambrosia075| NULL|but when you do i...|    1445040000|    true|NULL|
|0700026657| NULL|    4.0|I played it a whi...|07 27, 2015|A1JGAP0185YJI6|           travis| NULL|But in spite of t...|    1437955200|   false|NULL|
|0700026657| NULL|    3.0|            ok game.|02 23, 2015|A1YJWEXHQBWK2B|Vincent G. Mezera| NULL|         Three Stars|    1424649600|    true|NULL|
|0700026657| NULL|    2.0|found the game a ...|02 20, 2015|A2204E1TH211HT|       Grandma KR| NULL|        

In [ ]:
df = df.select("overall","reviewText")
#df.show(truncate=False)

In [ ]:
#Create a delta frame from the df and save to the hdfs
df.write.format("delta").mode("overwrite").save("/content/drive/MyDrive/GoogleColab")

In [ ]:
#fetch the delta table from hdfs
delta_df = spark.read.format("delta").load("/content/drive/MyDrive/GoogleColab")
delta_df.show(3,truncate=False)

+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|overall|reviewText                                                                                                                                                                                                                                                                                               |
+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|5.0    |This game is a bit hard to get the hang of, but when you do it's gr

In [ ]:
# Convert 'overall' to binary (0 or 1)
delta_df = delta_df.withColumn("overall", when(delta_df["overall"] < 3, 0).otherwise(1))
#delta_df.show(3,truncate=False)

In [ ]:
# Define a custom transformer to remove special characters
class RemoveSpecialCharactersTransformer(Transformer):
    def __init__(self, inputCol, outputCol):
        super(RemoveSpecialCharactersTransformer, self).__init__()
        self.inputCol = inputCol
        self.outputCol = outputCol

    def _transform(self, df: DataFrame) -> DataFrame:
#        print(f"Ran _transform with data {df[self.inputCol]}")
        return df.withColumn(self.outputCol, udf(remove_special_characters)(df[self.inputCol]))

# Define the UDF to remove special characters
def remove_special_characters(text):
    print(text, '\n')
    if type(text)==str:
        return re.sub(r'[^A-Za-z0-9\s]', '', text)

    return ""

# Create instances of the custom transformers
remove_special_chars = RemoveSpecialCharactersTransformer(inputCol="reviewText", outputCol="cleanText")


In [ ]:
# Tokenize the reviewText
tokenizer = Tokenizer(inputCol="cleanText", outputCol="token")

In [ ]:
# Define the StopWordsRemover
stop_words_remover = StopWordsRemover(inputCol="token", outputCol="words")

# Vectorize the filtered words using HashingTF
hashingTF = HashingTF(inputCol="words", outputCol="features")

In [ ]:
# Split the data into training and testing sets
train_data, test_data = delta_df.randomSplit([0.7, 0.3])

In [ ]:
# Define the Naive Bayes model
nb = NaiveBayes(featuresCol="features", labelCol="overall")

# Create a pipeline with Tokenizer, StopWordsRemover, HashingTF, and Naive Bayes
pipeline = Pipeline(stages=[remove_special_chars,tokenizer, stop_words_remover, hashingTF, nb])


In [ ]:
# Train the model
model = pipeline.fit(train_data)

In [ ]:
# Make predictions on the test data
predictions = model.transform(test_data)

# Evaluate the model using MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="overall", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)



Accuracy: 0.9117300008045268


In [ ]:
# make predictions on new reviewText
new_review_text = "good negative bad ouch terrible"
new_data = spark.createDataFrame([(new_review_text,)], ["reviewText"])
new_predictions = model.transform(new_data)

# Show the prediction for the new reviewText
new_predictions.select("reviewText", "prediction").show(truncate=False)

#repartitioning technique
#current_partitions= df.rdd.getNumPartitions()
#print(f"Current number of partitions: {current_partitions}")
"""desired_partition=4
#df_repartitioned=df.repartition(desired_partition)
#new_partitions=df_repartitioned.rdd.getNumPartitions()
#print(f"New number of partitions: {new_partitions}")"""

AttributeError: ignored

In [ ]:

#calculating the end time and duration of executing
"""end_time=time.time()
execution_time=end_time- start_time
print(f"Execution time: {execution_time} seconds")
"""
#salting
"""num_salt_buckets=40
df_salted=df.withColumn("salt", (rand()*num_salt_buckets).cast("int"))
df_salted_repartitionized=df_salted.repartition("salt")"""

# Unpersist the DataFrame, free up the memory
#df.unpersist()

# Stop the Spark session
#spark.stop()

In [ ]:
#spark = SparkSession.builder.master('local[*]').config("spark.driver.memory", "30g").appName('myProject_data_intensive').getOrCreate()


In [ ]:
import os

#Create empty pyspark dataframe
schema = StructType([StructField('overall', StringType(), True),
                     StructField('reviewText', StringType(), True)])
emptyRDD = spark.sparkContext.emptyRDD()
df = spark.createDataFrame(emptyRDD,schema)
#df.printSchema()
df.show()


dataPath="/content/drive/MyDrive/Colab Notebooks (1)/textData"

columns = ["overall", "reviewText"]
start1="overall: "
end1=".0,"
start2="reviewText: "
end2=", summary"

for filename in os.listdir(dataPath):
    print(filename)
    mystring = open(os.path.join(dataPath, filename)).read().replace('\n', '')
    ListReviewers = extraxtString(mystring,"overall:","overall:")

    myList=[]
    for i in range(0,len(ListReviewers)):
        s=ListReviewers[i]
        if "overall" and "reviewText" in s:
            string1 = s[s.find(start1)+len(start1):s.rfind(end1)]
            string2 = s[s.find(start2)+len(start2):s.rfind(end2)]
            myList.append([string1,string2])

    print(filename,"is structured.")
    #dataTemp = myList[0:1000]
    dataTemp = myList
    dataFrameTemp = spark.createDataFrame(dataTemp, columns)
    #dataFrameTemp.show()

    # concatinate data frames
    df = df.union(dataFrameTemp)


+-------+----------+
|overall|reviewText|
+-------+----------+
+-------+----------+

Video_Games_5_test.txt
Video_Games_5_test.txt is structured.
Arts_Crafts_and_Sewing_5_test.txt
Arts_Crafts_and_Sewing_5_test.txt is structured.


Py4JJavaError: ignored

In [ ]:
df.show()

+-------+--------------------+
|overall|          reviewText|
+-------+--------------------+
|      4|I've been using D...|
|      4|The demo is done ...|
|      5|If you've been wa...|
|      5|I've been creatin...|
|      5|I decided (after ...|
|      5|The video is well...|
|      5|I spent several h...|
|      5|I have had Dreamw...|
|      5|I have also taken...|
|      5|Even though I use...|
|      3|I waited to compl...|
|      5|As someone who ha...|
|      5|Adobe approved \A...|
|      5|This is a greate ...|
|      3|There are over 10...|
|      5|I am not an avid ...|
|      5|I am a long time ...|
|      4|The \Learn by Vid...|
|      4|The presenter her...|
|      4|This certified as...|
+-------+--------------------+
only showing top 20 rows



In [ ]:
#df.toPandas().to_csv('/content/drive/MyDrive/Colab Notebooks (1)/mycsv.csv')